# Código 1

In [ ]:
# Importação da biblioteca Pandas
import pandas as pd

In [ ]:
# Carregamento da base de dados de clientes
cust_df = pd.read_csv('customer_data.csv')

In [ ]:
# Carregamento da base de dados das transações dos clientes
cust_pymt_df= pd.read_csv('payment_data.csv')

In [ ]:
# Detalhando os tipos de dados da base de clientes
cust_df.info()

In [ ]:
# Detalhando os tipos de dados da base de transações
cust_pymt_df.info()

# Código 2

In [ ]:
# Removendo colunas que não serão utilizadas
cust_pymt_df.drop(columns=['prod_limit', 'update_date', 'report_date'], inplace=True)

# Código 3

In [ ]:
# Verificando quantos valores vazios cada campo possui na base de clientes
cust_df.isnull().sum()

In [ ]:
# Verificando quantos valores vazios cada campo possui na base de transações
cust_pymt_df.isnull().sum()

# Código 4

In [ ]:
# Apresentando a quantidade, média, desvio padrão, mínimo, quartis e máximo dos campos na base de clientes
cust_df.describe()

In [ ]:
# Apresentando a quantidade, média, desvio padrão, mínimo, quartis e máximo dos campos na base de transações
cust_pymt_df.describe()

# Código 5

In [ ]:
# Preenchendo com a média dos valores de fea_2 os registros em que este campo está vazio
cust_df['fea_2'].fillna(cust_df['fea_2'].mean(), inplace=True)

# Código 6

In [ ]:
# Removendo os registros em que há valores ausentes no campo highest_balance
cust_pymt_df.dropna(subset=['highest_balance'], inplace=True)

In [ ]:
# Resetando os índices do vetor para a que fique de 0 a N-1
cust_pymt_df.reset_index(drop=True, inplace=True)

# Código 7

In [ ]:
# Checando de há valores duplicados no campo id
cust_df['id'].duplicated().any()

# Código 8

In [ ]:
# Escolhendo os campos da base de clientes
cust_columns = ['fea_2', 'fea_4', 'fea_10', 'fea_11']

In [ ]:
# Listando os valores dos campos escolhidos na base de clientes em busca de outliers
for a in cust_columns:
    print(a, *cust_df[a].sort_values(ascending=True).tolist(), sep='\n')

In [ ]:
# Escolhendo os campos da base de transações
cust_pymt_columns = ['OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal', 'new_balance', 'highest_balance']

In [ ]:
# Listando os valores dos campos escolhidos na base de transações em busca de outliers
for a in cust_pymt_columns:
    print(a, *cust_pymt_df[a].sort_values(ascending=True).tolist(), sep='\n')

# Código 9

In [ ]:
# Mesclando as duas bases de dados utilizando a coluna "id" como chave em ambas
cust_df_mrg = pd.merge(cust_df, cust_pymt_df, left_on=['id'], right_on=['id'], how='inner')

# Removendo o campo "id" pois ele não será mais necessário
cust_df_mrg.drop(columns=['id'], inplace=True)

# Código 10

In [ ]:
# Importação da biblioteca de plotagem Seaborn
import seaborn as sns

In [ ]:
# Obtendo a quantidade total de itens 0 e 1 do campo "label"
items = cust_df_mrg['label'].value_counts()

In [ ]:
# Plotando o gráfico com a quantidade de cada uma das classificações
sns.barplot(x='Classificação', y='Quantidade', data=pd.DataFrame({'Classificação': items.index, 'Quantidade': items.values}))

# Código 11

In [ ]:
# Importação da biblioteca de plotagem Matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Definindo as variáveis categóricas
cat_features = ['fea_1', 'fea_3', 'fea_5', 'fea_6', 'fea_7', 'fea_9', 'prod_code']

In [ ]:
# Obtendo as correlações entre campos
correlations = cust_df_mrg.drop(columns=cat_features).corr()

In [ ]:
# Definindo as dimensões do mapa de calor
plt.figure(figsize=(14, 11))

# Plotando o mapa de calor
sns.heatmap(correlations, cmap=plt.cm.RdYlBu_r, annot=True, vmin=-1, vmax=1)

# Código 12

In [ ]:
# Importação do divisor de amostras para treino e teste
from sklearn.model_selection import train_test_split

# Extraindo os campos de features para o treinamento
x = cust_df_mrg.drop('label', axis=1)

# Extraindo a coluna com a classificação
y = cust_df_mrg['label']

# Separando as amostras para treinamento e teste, numa proportção de 80%/20% respectivamente
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Código 13

In [ ]:
# Importação do classificador de árvore de decisão
from sklearn.tree import DecisionTreeClassifier

# Instanciando o classificador
tree = DecisionTreeClassifier()

# Treinando o classificador
tree = tree.fit(x_train, y_train)

# Código 14

In [ ]:
# Exibindo os campos e seus graus de importância para o treinamento
pd.Series(tree.feature_importances_, index=x_train.columns).sort_values(ascending=True).plot(kind='barh', figsize=(15, 6))

# Código 15

In [ ]:
# Executando o teste de previsão com as amostras de teste
tree_result = tree.predict(x_test)

# Importação da matriz de confusão
from sklearn.metrics import confusion_matrix

# Exibição da matriz de confusão
print(confusion_matrix(y_test, tree_result))

# Código 16

In [ ]:
# Importação do cálculo de métricas
from sklearn.metrics import classification_report

# Exibindo as métricas
print(classification_report(y_test, tree_result, digits=4))

# Código 17

In [ ]:
# Importação do classificador de Naïve Bayes
from sklearn.naive_bayes import GaussianNB

# Instanciando o classificador
naive_bayes = GaussianNB()

# Treinando o classificador
naive_bayes = naive_bayes.fit(x_train, y_train)

# Código 18

In [ ]:
# Executando o teste de previsão com as amostras de teste
naive_bayes_result = naive_bayes.predict(x_test)

# Exibição da matriz de confusão
print(confusion_matrix(y_test, naive_bayes_result))

# Código 19

In [ ]:
# Exibindo as métricas
print(classification_report(y_test, naive_bayes_result, digits=4))

# Código 20

In [ ]:
# Importação do classificador de regressão logística
from sklearn.linear_model import LogisticRegression

# Instanciando o classificador
logistic = LogisticRegression()

# Treinando o classificador
logistic = logistic.fit(x_train, y_train)

# Código 21

In [ ]:
# Executando o teste de previsão com as amostras de teste
logistic_result = logistic.predict(x_test)

# Exibição da matriz de confusão
print(confusion_matrix(y_test, logistic_result))

# Código 22

In [ ]:
# Exibindo as métricas
print(classification_report(y_test, logistic_result, digits=4))

# Código 23

In [ ]:
# Importação do classificador de SVM
from sklearn import svm

# Instanciando o classificador
svm = svm.SVC()

# Treinando o classificador
svm = svm.fit(x_train, y_train)

# Código 24

In [ ]:
# Executando o teste de previsão com as amostras de teste
svm_result = svm.predict(x_test)

# Exibição da matriz de confusão
print(confusion_matrix(y_test, svm_result))

# Código 25

In [ ]:
# Exibindo as métricas
print(classification_report(y_test, svm_result, digits=4))

# Código 26

In [ ]:
# Importação do classificador de rede neural artificial
from sklearn.neural_network import MLPClassifier

# Instanciando o classificador
ann = MLPClassifier()

# Treinando o classificador
ann = ann.fit(x_train, y_train)

# Código 27

In [ ]:
# Executando o teste de previsão com as amostras de teste
ann_result = ann.predict(x_test)

# Exibição da matriz de confusão
print(confusion_matrix(y_test, ann_result))

# Código 28

In [ ]:
# Exibindo as métricas
print(classification_report(y_test, ann_result, digits=4))

# Código 29

In [ ]:
# Importação do classificador de floresta aleatória
from sklearn.ensemble import RandomForestClassifier

# Instanciando o classificador com 1000000 árvores
forest = RandomForestClassifier(n_estimators = 1000000)

# Treinando o classificador
forest = forest.fit(x_train, y_train)

# Código 30

In [ ]:
# Executando o teste de previsão com as amostras de teste
forest_result = forest.predict(x_test)

# Exibição da matriz de confusão
print(confusion_matrix(y_test, forest_result))

# Código 31

In [ ]:
# Exibindo as métricas
print(classification_report(y_test, forest_result, digits=4))